In [13]:
import numpy as np 
import pandas as pd
import os
import re
import enchant

from tqdm import tqdm

from cnn_fasttext import CNN_model

6926
Liste de médicaments regroupant les libéllés ATC et les dénominations de spécialité,  de taille: 8275
Sample of medicament names: 
 ['ubistesin adrenalinee', 'forene,', 'sevorane,', 'chirocaïne', 'duodopa', 'synagis', 'kaletra', 'humira', 'norvir', 'viekirax']
Size of the vocab 10141


ValueError: not enough values to unpack (expected 2, got 0)

In [2]:
NB_CATEGORIES = 52
TRAINABLE = False

In [3]:
dataFolder = '../posos-data-challenge/challenge_data'
medicament_list_path = '../external doc/medicaments_france.xls'
xPath = os.path.join(dataFolder, 'input_train.csv')
yPath = os.path.join(dataFolder, 'challenge_output_data_training_file_predict_the_expected_answer.csv')
model = CNN_model(nbCategories=NB_CATEGORIES, trainable = TRAINABLE)
# Loading, parsing and spliting training and testing data
x = pd.read_csv(xPath, delimiter=';', usecols=[1]).values.ravel()
y = pd.read_csv(yPath, delimiter=';', usecols=[1]).values.ravel()

# keras tokenizer gives all informations about our vocabulary
model.preprocess(x)

x_vocab  = list(model.tokenizer.word_index.keys())
print('Size of the vocab', len(x_vocab))

Size of the vocab 10806


In [4]:
d = enchant.Dict('fr_FR')

## Try to include knowledge about the medicaments

In [5]:
medic_db = pd.read_excel(medicament_list_path)

In [6]:
MEDICAMENTS = []
for m in medic_db['Dénomination spécialité']:
    med = []
    for w in m.split():
        if w.lower()!=w:
            med.append(w)
    med = ' '.join(med)
    if len(med)!=0:
        med = med.lower()
        if med not in MEDICAMENTS:
            MEDICAMENTS.append(med.lower())
print(len(MEDICAMENTS))
for m in medic_db['Libellé ATC']:
    med = m.split()[0].lower()
    if med not in MEDICAMENTS:
        MEDICAMENTS.append(med)
print('Liste de médicaments regroupant les libéllés ATC et les dénominations de spécialité,  de taille: {}'.format(len(MEDICAMENTS)))
print('Sample of medicament names: \n', MEDICAMENTS[:10])
# add the medicaments to the dictionnary:
for w in MEDICAMENTS:
    d.add(w)

6926
Liste de médicaments regroupant les libéllés ATC et les dénominations de spécialité,  de taille: 8275
Sample of medicament names: 
 ['ubistesin adrenalinee', 'forene,', 'sevorane,', 'chirocaïne', 'duodopa', 'synagis', 'kaletra', 'humira', 'norvir', 'viekirax']


In [7]:
x_oov = [w for w in x_vocab if d.check(w) == False]
x_corrected = []

verbose = False

# use enchant to correct spelling errors (include medicaments that we inserted at the initialization of spell_dict)
common_mistakes = ['ème', 'eme', 'éme', 'ans', 'an', 'années', 'année', 'h', 'heure', 'mg', 'g', 'jrs', 'jours', 'min', 'mn', 'jour', 'kg', 'cm']
nb_re = re.compile('(\d+)(\S+)')
for w in tqdm(x_oov):
    nb = nb_re.search(w)
    if nb!= None:
        if nb.group(2) in common_mistakes:
            if verbose:
                print('catch numerical spelling error' + nb.group())
            w = w.replace(nb.group(1), nb.group(1) + ' ')
    # disgard other numerical expression that we don't want to clean
    if re.search('\d+', w) == None:
        corrected_w = d.suggest(w)[0]
        if verbose:
            print("corrected " + w + ' in ' + corrected_w)
        w = corrected_w
    x_corrected.append(w)
    
correct_dict = dict(zip(x_oov, x_corrected))

100%|██████████| 3814/3814 [13:07<00:00,  4.84it/s]


In [8]:
print('nbre de mots erronés', len(x_oov), " soit une proportion de  ", len(x_oov)/len(x_vocab), "mots absents de pyenchant dictionnary.")
print('sample of absent words:', x_oov[:10])

nbre de mots erronés 3814  soit une proportion de   0.3529520636683324 mots absents de pyenchant dictionnary.
sample of absent words: ['aurrais', 'coryzalia', 'acupan', 'havlane', 'beta', '22juillet', 'bromokin', 'climaxol', 'disgestions', 'hepatite']


In [9]:
correct_dict

{'aurrais': 'aurais',
 "l'euphytose": 'l’Euphrosyne',
 'anxio': 'anxiogène',
 'relaxis': 'relais',
 'ixprim37': 'ixprim37',
 'acupan': 'occupante',
 'tagagisme': 'tabagisme',
 'havlane': 'havane',
 'beta': 'bêta',
 'schoen': 'Schoten',
 'voi': 'coi',
 '22juillet': '22juillet',
 'kaki23': 'kaki23',
 'bromokin': 'bromoxynil',
 'antitranspirant': 'anti transpirant',
 'chque': 'chue',
 'commençer': 'commencer',
 'preparation': 'préparation',
 'alpraz': 'alcarazas',
 'disgestions': 'digestions',
 'lamisilate': 'islamiste',
 'reponse': 'réponse',
 'avc': 'abc',
 'purinéthol': 'allopurinol',
 'l’ibiprofene': 'ibuprofène',
 "l'ipeca": 'l’ipéca',
 'prazinil': 'pralin',
 'stérogyl': 'stérol',
 'aprés': 'après',
 'enchainant': 'enchaînant',
 'l’inexium': 'ununhexium',
 'oedemes': 'demeures',
 'hémorroidaire': 'hémorroïdaire',
 'recapture': 'capture',
 'peti': 'pite',
 'alccol': 'alcool',
 'troisème': 'troisième',
 'dtp': 'ftp',
 'cheri': 'chéri',
 'desagréables': 'désagréables',
 "l'avonex": 'l’Y

In [11]:
import csv
w = csv.writer(open('../posos-data-challenge/challenge_data/corrections.csv', "w"))
for key, val in correct_dict.items():
    w.writerow([key, val])

In [10]:
# input the name of the medicaments
# On peut tenter d'introduire le wordembedding du médicament dans le 
"""implement a spelling correcter based on pyenchant, to be included in the preprocessingsteps of our model, after tokenization and medicament targetting.
input:  q, a question to correct (as a list of token)
        verbose, boolean
        medicaments, the list of medicaments commercialised in France
output the corrected question 
"""
def spell_correction(q, verbose = False, medicaments = []):
    corrected_q = []
    # common mistake with numbers
    for w in q:        
    # disgard other numerical expression that we don't want to clean
        if re.search('\d', w) == None:
            if d.check(w) == False:
                w = d.suggest(w)[0]
        corrected_q.append(w)
    return corrected_q


COMMON_NB_MISTAKES = ['ème', 'eme', 'éme', 'ans', 'an', 'années', 'année', 'h', 'heure', 'mg', 'g', 'jrs', 'jours', 'min', 'mn', 'jour', 'kg', 'cm']
def regex_nb_correction(q, verbose = False, common_nb_mistakes = COMMON_NB_MISTAKES):
    corrected_q = []
    for w in q:
        # match numerical errors like '5jours' and correct to '5 jours'
        nb = nb_re.search(w)
        if nb!= None:
            if nb.group(2) in common_mistake:
                if verbose:
                    print('catch numerical spelling error' + nb.group())
                w = w.replace(nb.group(1), nb.group(1) + ' ')
        corrected_q.append(w)
    return corrected_q

In [ ]:
rex